In [1]:
import numpy as np
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.wcs import WCS
import os
from scipy import optimize, stats, interpolate
from scipy.integrate import quad
import argparse
import time
import multiprocessing as mp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import cm
import sys
import pandas as pd
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
import healpy as hp
from copy import copy, deepcopy
# sys.path.append('BatML/')
import logging, traceback
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
NITRATES_path = '/home/jimmy/local/NITRATES/NITRATES/'
%cd {NITRATES_path}

/home/jimmy/local/NITRATES/NITRATES


In [3]:
# if the response directories are all put in the same directory
# and given as an env variable (like in this cell), then config.py will 
# be able to find them correctly as long as they keep the original names 
# (Ex: ray trace directotry is still called 'ray_traces_detapp_npy')
# These paths can also just be hardcoded into config.py
# most of these paths can also be given directly as an argument to 
# some analysis objects, like Source_Model_InOutFoV()
os.environ['NITRATES_RESP_DIR'] = '/home/jimmy/local/NITRATES/data/'

In [4]:
from config import rt_dir
from ray_trace_funcs import RayTraces
from event2dpi_funcs import det2dpis, mask_detxy
from flux_models import Cutoff_Plaw_Flux, Plaw_Flux, get_eflux_from_model
from models import Source_Model_InOutFoV, Bkg_Model_wFlatA,\
                CompoundModel, Point_Source_Model_Binned_Rates, im_dist
from ray_trace_funcs import RayTraces
from sqlite_funcs import get_conn
from dbread_funcs import get_info_tab, get_twinds_tab
from hp_funcs import ang_sep
from coord_conv_funcs import theta_phi2imxy, imxy2theta_phi, convert_imxy2radec,\
                            convert_radec2thetaphi, convert_radec2imxy
from do_llh_inFoV4realtime2 import parse_bkg_csv
from LLH import LLH_webins
from minimizers import NLLH_ScipyMinimize_Wjacob

In [5]:
%matplotlib inline

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [6]:
ebins0 = np.array([15.0, 24.0, 35.0, 48.0, 64.0])
ebins0 = np.append(ebins0, np.logspace(np.log10(84.0), np.log10(500.0), 5+1))[:-1]
ebins0 = np.round(ebins0, decimals=1)[:-1]
ebins1 = np.append(ebins0[1:], [350.0])
nebins = len(ebins0)
print("Nebins: ", nebins)
print("ebins0: ", ebins0)
print("ebins1: ", ebins1)

# work_dir = '/storage/work/jjd330/local/bat_data/realtime_workdir/F646018360/'
work_dir = os.path.join(NITRATES_path, 'F646018360')
conn = get_conn(os.path.join(work_dir,'results.db'))
info_tab = get_info_tab(conn)
trigger_time = info_tab['trigtimeMET'][0]
print('trigger_time: ', trigger_time)

evfname = os.path.join(work_dir,'filter_evdata.fits')
ev_data = fits.open(evfname)[1].data
GTI_PNT = Table.read(evfname, hdu='GTI_POINTING')
GTI_SLEW = Table.read(evfname, hdu='GTI_SLEW')
dmask = fits.open(os.path.join(work_dir,'detmask.fits'))[0].data
attfile = fits.open(os.path.join(work_dir,'attitude.fits'))[1].data

att_ind = np.argmin(np.abs(attfile['TIME'] - trigger_time))
att_quat = attfile['QPARAM'][att_ind]
print('attitude quaternion: ', att_quat)

ndets = np.sum(dmask==0)
print("Ndets: ", np.sum(dmask==0))

t_end = trigger_time + 1e3
t_start = trigger_time - 1e3
mask_vals = mask_detxy(dmask, ev_data)
bl_dmask = (dmask==0.)

bl_ev = (ev_data['EVENT_FLAGS']<1)&\
        (ev_data['ENERGY']<=500.)&(ev_data['ENERGY']>=14.)&\
        (mask_vals==0.)&(ev_data['TIME']<=t_end)&\
        (ev_data['TIME']>=t_start)

print("Nevents: ",np.sum(bl_ev))
ev_data0 = ev_data[bl_ev]

('Nebins: ', 9)
('ebins0: ', array([ 15. ,  24. ,  35. ,  48. ,  64. ,  84. , 120. , 171.5, 245. ]))
('ebins1: ', array([ 24. ,  35. ,  48. ,  64. ,  84. , 120. , 171.5, 245. , 350. ]))
('trigger_time: ', 646018383.1787)
('attitude quaternion: ', array([-0.03597053,  0.2345147 , -0.64420835,  0.72712074]))
('Ndets: ', 14932)
('Nevents: ', 1367885)


In [7]:
ra, dec = 233.117, -26.213 
theta, phi = convert_radec2thetaphi(ra, dec, att_quat)
print(theta, phi)
imx, imy = convert_radec2imxy(ra, dec, att_quat)
print(imx, imy)

(38.54132137017975, 137.65241966813443)
(-0.5887551341212707, -0.5366203642198198)


In [8]:
flux_params = {'A':1.0, 'gamma':0.5, 'Epeak':1e2}
flux_mod = Cutoff_Plaw_Flux(E0=100.0)

In [9]:
# use rt_dir from config.py or
# rt_dir = '/path/to/ray_traces_detapp_npy/'
rt_obj = RayTraces(rt_dir)

INFO:root:0.02
INFO:root:0.002


In [10]:
rt = rt_obj.get_intp_rt(imx, imy)
print(np.shape(rt))

DEBUG:root:opening file: fwd_ray_trace_-0.60000_-0.54000_-0.58000_-0.52000_.npy
DEBUG:root:nbytes_total=95789408.0
(173, 286)


In [11]:
%%time
# will by default use the resp directories from config.py
sig_mod = Source_Model_InOutFoV(flux_mod, [ebins0,ebins1], bl_dmask,\
                                rt_obj, use_deriv=True)
# or the paths can be given
# resp_tab_dname = '/path/to/resp_tabs_ebins/'
# hp_flor_resp_dname = '/path/to/hp_flor_resps/'
# comp_flor_resp_dname = '/path/to/comp_flor_resps/'
# sig_mod = Source_Model_InOutFoV(flux_mod, [ebins0,ebins1], bl_dmask,\
#                                 rt_obj, use_deriv=True,\
#                                 resp_tab_dname=resp_tab_dname,\
#                                 hp_flor_resp_dname=hp_flor_resp_dname,\
#                                 comp_flor_resp_dname=comp_flor_resp_dname)

CPU times: user 2.92 ms, sys: 0 ns, total: 2.92 ms
Wall time: 1.72 ms


In [12]:
%%time
sig_mod.set_theta_phi(theta, phi)

INFO:root:Making new response object
(0.0, 30.48, -14.117)
DEBUG:numba.interpreter:label 0:
    wall_bnds = arg(0, name=wall_bnds)       ['wall_bnds']
    $const0.2 = const(int, 0)                ['$const0.2']
    $0.3 = getitem(index=$const0.2, value=wall_bnds) ['$0.3', '$const0.2', 'wall_bnds']
    x0 = $0.3                                ['$0.3', 'x0']
    $const0.5 = const(int, 1)                ['$const0.5']
    $0.6 = getitem(index=$const0.5, value=wall_bnds) ['$0.6', '$const0.5', 'wall_bnds']
    x1 = $0.6                                ['$0.6', 'x1']
    $const0.8 = const(int, 2)                ['$const0.8']
    $0.9 = getitem(index=$const0.8, value=wall_bnds) ['$0.9', '$const0.8', 'wall_bnds']
    x2 = $0.9                                ['$0.9', 'x2']
    $0.12 = x1 - x0                          ['$0.12', 'x0', 'x1']
    v01 = $0.12                              ['$0.12', 'v01']
    $0.15 = x2 - x0                          ['$0.15', 'x0', 'x2']
    v02 = $0.15                 

DEBUG:numba.interpreter:label 0:
    x = arg(0, name=x)                       ['x']
    $const_Ellipsis = const(ellipsis, Ellipsis) ['$const_Ellipsis']
    $const_0 = const(int, 0)                 ['$const_0']
    $const0.2 = build_tuple(items=[Var($const_Ellipsis, arraymath.py:4146), Var($const_0, arraymath.py:4146)]) ['$const0.2', '$const_0', '$const_Ellipsis']
    $0.3 = getitem(index=$const0.2, value=x) ['$0.3', '$const0.2', 'x']
    x0 = $0.3                                ['$0.3', 'x0']
    $const_Ellipsis.1 = const(ellipsis, Ellipsis) ['$const_Ellipsis.1']
    $const_1 = const(int, 1)                 ['$const_1']
    $const0.5 = build_tuple(items=[Var($const_Ellipsis.1, arraymath.py:4147), Var($const_1, arraymath.py:4147)]) ['$const0.5', '$const_1', '$const_Ellipsis.1']
    $0.6 = getitem(index=$const0.5, value=x) ['$0.6', '$const0.5', 'x']
    x1 = $0.6                                ['$0.6', 'x1']
    $0.8 = getattr(attr=shape, value=x)      ['$0.8', 'x']
    $const0.9 = const

DEBUG:numba.interpreter:label 0:
    lst = arg(0, name=lst)                   ['lst']
    value = arg(1, name=value)               ['value']
    jump 3                                   []
label 3:
    $3.2 = getiter(value=lst)                ['$3.2', 'lst']
    $phi7.1 = $3.2                           ['$3.2', '$phi7.1']
    jump 7                                   []
label 7:
    $7.2 = iternext(value=$phi7.1)           ['$7.2', '$phi7.1']
    $7.3 = pair_first(value=$7.2)            ['$7.2', '$7.3']
    $7.4 = pair_second(value=$7.2)           ['$7.2', '$7.4']
    $phi10.1 = $7.3                          ['$7.3', '$phi10.1']
    $phi32.1 = $7.3                          ['$7.3', '$phi32.1']
    $phi32.2 = $phi7.1                       ['$phi32.2', '$phi7.1']
    branch $7.4, 10, 32                      ['$7.4']
label 10:
    elem = $phi10.1                          ['$phi10.1', 'elem']
    $10.4 = elem == value                    ['$10.4', 'elem', 'value']
    branch $10.4, 25, 7    

DEBUG:numba.interpreter:label 0:
    src_ndim = arg(0, name=src_ndim)         ['src_ndim']
    src_shape = arg(1, name=src_shape)       ['src_shape']
    dest_ndim = arg(2, name=dest_ndim)       ['dest_ndim']
    dest_shape = arg(3, name=dest_shape)     ['dest_shape']
    $0.3 = src_ndim > dest_ndim              ['$0.3', 'dest_ndim', 'src_ndim']
    branch $0.3, 12, 16                      ['$0.3']
label 12:
    $const12.1 = const(int, 0)               ['$const12.1']
    $12.2 = cast(value=$const12.1)           ['$12.2', '$const12.1']
    return $12.2                             ['$12.2']
label 16:
    $const16.1 = const(int, 0)               ['$const16.1']
    src_index = $const16.1                   ['$const16.1', 'src_index']
    $16.4 = dest_ndim - src_ndim             ['$16.4', 'dest_ndim', 'src_ndim']
    dest_index = $16.4                       ['$16.4', 'dest_index']
    jump 32                                  []
label 32:
    jump 35                                  []
label 

DEBUG:numba.interpreter:label 0:
    _0_10 = arg(0, name=_0_10)               ['_0_10']
    v = arg(1, name=v)                       ['v']
    $0.3 = v / _0_10                         ['$0.3', '_0_10', 'v']
    $0.4 = cast(value=$0.3)                  ['$0.3', '$0.4']
    return $0.4                              ['$0.4']

DEBUG:numba.interpreter:label 0:
    x0 = arg(0, name=x0)                     ['x0']
    x1 = arg(1, name=x1)                     ['x1']
    $0.3 = x1 - x0                           ['$0.3', 'x0', 'x1']
    $0.4 = cast(value=$0.3)                  ['$0.3', '$0.4']
    return $0.4                              ['$0.4']

DEBUG:numba.interpreter:label 0:
    x0 = arg(0, name=x0)                     ['x0']
    x2 = arg(1, name=x2)                     ['x2']
    $0.3 = x2 - x0                           ['$0.3', 'x0', 'x2']
    $0.4 = cast(value=$0.3)                  ['$0.3', '$0.4']
    return $0.4                              ['$0.4']



...

DEBUG:numba.interpreter:label 0:
    a = arg(0, name=a)                       ['a']
    b = arg(1, name=b)                       ['b']
    $0.1 = freevar(tp: float64)              ['$0.1']
    $const0.2 = const(int, 1)                ['$const0.2']
    $0.3 = call $0.1($const0.2, kws=[], args=[Var($const0.2, numbers.py:225)], func=$0.1, vararg=None) ['$0.1', '$0.3', '$const0.2']
    r = $0.3                                 ['$0.3', 'r']
    $0.4 = freevar(tp: float64)              ['$0.4']
    $0.6 = call $0.4(a, kws=[], args=[Var(a, numbers.py:225)], func=$0.4, vararg=None) ['$0.4', '$0.6', 'a']
    a.1 = $0.6                               ['$0.6', 'a.1']
    $const0.8 = const(int, 0)                ['$const0.8']
    $0.9 = b < $const0.8                     ['$0.9', '$const0.8', 'b']
    branch $0.9, 36, 147                     ['$0.9']
label 36:
    $36.1 = global(True: True)               ['$36.1']
    invert = $36.1                           ['$36.1', 'invert']
    $36.3 = unary(val

DEBUG:numba.interpreter:label 0:
    x0 = arg(0, name=x0)                     ['x0']
    y0 = arg(1, name=y0)                     ['y0']
    R = arg(2, name=R)                       ['R']
    zmin = arg(3, name=zmin)                 ['zmin']
    zmax = arg(4, name=zmax)                 ['zmax']
    gam_theta = arg(5, name=gam_theta)       ['gam_theta']
    gam_phi = arg(6, name=gam_phi)           ['gam_phi']
    batx = arg(7, name=batx)                 ['batx']
    baty = arg(8, name=baty)                 ['baty']
    batz = arg(9, name=batz)                 ['batz']
    $0.1 = global(np: <module 'numpy' from '/home/jimmy/miniconda3/envs/nitrates_py27/lib/python2.7/site-packages/numpy/__init__.pyc'>) ['$0.1']
    $0.2 = getattr(attr=zeros_like, value=$0.1) ['$0.1', '$0.2']
    $0.4 = call $0.2(batx, kws=[], args=[Var(batx, Polygons.py:118)], func=$0.2, vararg=None) ['$0.2', '$0.4', 'batx']
    dists = $0.4                             ['$0.4', 'dists']
    $const0.6 = const(int, 0)     

DEBUG:numba.interpreter:label 0:
    xs = arg(0, name=xs)                     ['xs']
    ys = arg(1, name=ys)                     ['ys']
    zs = arg(2, name=zs)                     ['zs']
    rot_mat = arg(3, name=rot_mat)           ['rot_mat']
    $0.1 = global(np: <module 'numpy' from '/home/jimmy/miniconda3/envs/nitrates_py27/lib/python2.7/site-packages/numpy/__init__.pyc'>) ['$0.1']
    $0.2 = getattr(attr=zeros_like, value=$0.1) ['$0.1', '$0.2']
    $0.4 = call $0.2(xs, kws=[], args=[Var(xs, shield_structure.py:46)], func=$0.2, vararg=None) ['$0.2', '$0.4', 'xs']
    x_news = $0.4                            ['$0.4', 'x_news']
    $0.5 = global(np: <module 'numpy' from '/home/jimmy/miniconda3/envs/nitrates_py27/lib/python2.7/site-packages/numpy/__init__.pyc'>) ['$0.5']
    $0.6 = getattr(attr=zeros_like, value=$0.5) ['$0.5', '$0.6']
    $0.8 = call $0.6(xs, kws=[], args=[Var(xs, shield_structure.py:46)], func=$0.6, vararg=None) ['$0.6', '$0.8', 'xs']
    y_news = $0.8             

DEBUG:root:Openning file: resp_by_sand_hpind_46.fits
DEBUG:numba.interpreter:label 0:
    sand_imgs = arg(0, name=sand_imgs)       ['sand_imgs']
    pha_bins0 = arg(1, name=pha_bins0)       ['pha_bins0']
    pha_bins1 = arg(2, name=pha_bins1)       ['pha_bins1']
    new_pha_bins0 = arg(3, name=new_pha_bins0) ['new_pha_bins0']
    new_pha_bins1 = arg(4, name=new_pha_bins1) ['new_pha_bins1']
    $0.2 = getattr(attr=size, value=new_pha_bins0) ['$0.2', 'new_pha_bins0']
    Nphabins_new = $0.2                      ['$0.2', 'Nphabins_new']
    $0.4 = getattr(attr=shape, value=sand_imgs) ['$0.4', 'sand_imgs']
    $const0.5 = const(int, 0)                ['$const0.5']
    $0.6 = getitem(index=$const0.5, value=$0.4) ['$0.4', '$0.6', '$const0.5']
    NphotonEs = $0.6                         ['$0.6', 'NphotonEs']
    $const0.8 = const(int, 16)               ['$const0.8']
    $const0.9 = const(int, 16)               ['$const0.9']
    $0.11 = build_tuple(items=[Var(NphotonEs, response.py:1428), Var

DEBUG:numba.interpreter:label 0:
    e0 = arg(0, name=e0)                     ['e0']
    e1 = arg(1, name=e1)                     ['e1']
    pha_bins0 = arg(2, name=pha_bins0)       ['pha_bins0']
    pha_bins1 = arg(3, name=pha_bins1)       ['pha_bins1']
    $0.3 = pha_bins0 >= e0                   ['$0.3', 'e0', 'pha_bins0']
    $0.6 = pha_bins1 <= e1                   ['$0.6', 'e1', 'pha_bins1']
    $0.7 = $0.3 & $0.6                       ['$0.3', '$0.6', '$0.7']
    $0.8 = cast(value=$0.7)                  ['$0.7', '$0.8']
    return $0.8                              ['$0.8']

DEBUG:numba.interpreter:label 0:
    e0 = arg(0, name=e0)                     ['e0']
    pha_bins0 = arg(1, name=pha_bins0)       ['pha_bins0']
    pha_bins1 = arg(2, name=pha_bins1)       ['pha_bins1']
    $0.3 = pha_bins0 < e0                    ['$0.3', 'e0', 'pha_bins0']
    $0.6 = pha_bins1 > e0                    ['$0.6', 'e0', 'pha_bins1']
    $0.7 = $0.3 & $0.6                       ['$0.3', '$0.6', '

In [13]:
from config import solid_angle_dpi_fname
# or give the path directly
# solid_angle_dpi_fname = '/path/to/solid_angle_dpi.npy'
bkg_fname = os.path.join(work_dir,'bkg_estimation.csv')
solid_ang_dpi = np.load(solid_angle_dpi_fname)
bkg_df, bkg_name, PSnames, bkg_mod, ps_mods = parse_bkg_csv(bkg_fname, solid_ang_dpi,\
                    ebins0, ebins1, bl_dmask, rt_dir)
bkg_mod.has_deriv = False
bkg_mod_list = [bkg_mod]
Nsrcs = len(ps_mods)
if Nsrcs > 0:
    bkg_mod_list += ps_mods
    for ps_mod in ps_mods:
        ps_mod.has_deriv = False
    bkg_mod = CompoundModel(bkg_mod_list)
tmid = trigger_time
bkg_row = bkg_df.iloc[np.argmin(np.abs(tmid - bkg_df['time']))]
bkg_params = {pname:bkg_row[pname] for pname in\
            bkg_mod.param_names}
bkg_name = bkg_mod.name
bkg_params

['4U 1700-377', 'GRO J1655-40', 'GX 339-4', 'Sco X-1']
INFO:root:0.02
INFO:root:0.002
DEBUG:root:opening file: fwd_ray_trace_-0.10000_-0.48000_-0.08000_-0.46000_.npy
DEBUG:root:nbytes_total=95789408.0
DEBUG:root:opening file: fwd_ray_trace_-0.12000_-0.54000_-0.10000_-0.52000_.npy
DEBUG:root:nbytes_total=191578816.0
DEBUG:root:opening file: fwd_ray_trace_0.00000_-0.70000_0.02000_-0.68000_.npy
DEBUG:root:nbytes_total=287368224.0
DEBUG:root:opening file: fwd_ray_trace_-0.50000_-0.22000_-0.48000_-0.20000_.npy
DEBUG:root:nbytes_total=383157632.0


/home/jimmy/miniconda3/envs/nitrates_py27/lib/python2.7/site-packages/numpy/core/fromnumeric.py:51: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  return getattr(obj, method)(*args, **kwds)


{'4U 1700-377_imx': -0.09814853057709713,
 '4U 1700-377_imy': -0.4742076074486664,
 '4U 1700-377_rate_0': 0.012490135921897544,
 '4U 1700-377_rate_1': 0.00724252648862388,
 '4U 1700-377_rate_2': 0.004529564586462892,
 '4U 1700-377_rate_3': 0.0016876913514016756,
 '4U 1700-377_rate_4': 0.0013255091059552946,
 '4U 1700-377_rate_5': 0.0012956105576278066,
 '4U 1700-377_rate_6': 0.0005834459169539205,
 '4U 1700-377_rate_7': 5e-08,
 '4U 1700-377_rate_8': 0.00014013200316638987,
 'Background_bkg_rate_0': 0.09137032207011836,
 'Background_bkg_rate_1': 0.0661578002239374,
 'Background_bkg_rate_2': 0.040089856902610514,
 'Background_bkg_rate_3': 0.039491993420780136,
 'Background_bkg_rate_4': 0.03467802668352813,
 'Background_bkg_rate_5': 0.0351076584048365,
 'Background_bkg_rate_6': 0.03360249200395507,
 'Background_bkg_rate_7': 0.024861082684787414,
 'Background_bkg_rate_8': 0.01763994761450761,
 'Background_flat_0': 0.0,
 'Background_flat_1': 0.0,
 'Background_flat_2': 0.18596326494289767,
 

In [14]:
pars_ = {}
pars_['Signal_theta'] = theta
pars_['Signal_phi'] = phi
for pname,val in bkg_params.iteritems():
    pars_[bkg_name+'_'+pname] = val
for pname,val in flux_params.iteritems():
    pars_['Signal_'+pname] = val
pars_

{'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_imx': -0.09814853057709713,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_imy': -0.4742076074486664,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_0': 0.012490135921897544,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_1': 0.00724252648862388,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_2': 0.004529564586462892,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_3': 0.0016876913514016756,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_4': 0.0013255091059552946,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_5': 0.0012956105576278066,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_6': 0.0005834459169539205,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_7': 5e-08,
 'Background+4U 1700-377+GRO J1655-40+G

In [15]:
comp_mod = CompoundModel([bkg_mod, sig_mod])

sig_miner = NLLH_ScipyMinimize_Wjacob('')

sig_llh_obj = LLH_webins(ev_data0, ebins0, ebins1, bl_dmask, has_err=True)

sig_llh_obj.set_model(comp_mod)

sig_miner.set_llh(sig_llh_obj)

In [16]:
fixed_pnames = pars_.keys()
fixed_vals = pars_.values()
trans = [None for i in range(len(fixed_pnames))]
sig_miner.set_trans(fixed_pnames, trans)
sig_miner.set_fixed_params(fixed_pnames, values=fixed_vals)
sig_miner.set_fixed_params(['Signal_A'], fixed=False)

In [17]:
%%time
flux_params['gamma'] = 0.8
flux_params['Epeak'] = 350.0
sig_mod.set_flux_params(flux_params)

DEBUG:numba.interpreter:label 0:
    resp_dpi = arg(0, name=resp_dpi)         ['resp_dpi']
    photon_fluxes = arg(1, name=photon_fluxes) ['photon_fluxes']
    $0.2 = getattr(attr=shape, value=resp_dpi) ['$0.2', 'resp_dpi']
    $0.6 = exhaust_iter(count=3, value=$0.2) ['$0.2', '$0.6']
    $0.3 = static_getitem(index=0, value=$0.6, index_var=None) ['$0.3', '$0.6']
    $0.4 = static_getitem(index=1, value=$0.6, index_var=None) ['$0.4', '$0.6']
    $0.5 = static_getitem(index=2, value=$0.6, index_var=None) ['$0.5', '$0.6']
    ndets = $0.3                             ['$0.3', 'ndets']
    NphotonEs = $0.4                         ['$0.4', 'NphotonEs']
    Nphabins = $0.5                          ['$0.5', 'Nphabins']
    $0.7 = global(np: <module 'numpy' from '/home/jimmy/miniconda3/envs/nitrates_py27/lib/python2.7/site-packages/numpy/__init__.pyc'>) ['$0.7']
    $0.8 = getattr(attr=zeros, value=$0.7)   ['$0.7', '$0.8']
    $0.11 = build_tuple(items=[Var(ndets, response.py:1110), Var(Nphabi

In [18]:
t0 = trigger_time - 0.512
t1 = t0 + 2.048
sig_llh_obj.set_time(t0, t1)

In [19]:
%%time
pars, nllh, res = sig_miner.minimize()

DEBUG:numba.interpreter:label 0:
    mu = arg(0, name=mu)                     ['mu']
    sig = arg(1, name=sig)                   ['sig']
    N = arg(2, name=N)                       ['N']
    size = arg(3, name=size)                 ['size']
    $const0.1 = const(float, 0.0)            ['$const0.1']
    llh_ = $const0.1                         ['$const0.1', 'llh_']
    jump 6                                   []
label 6:
    jump 9                                   []
label 9:
    $9.1 = global(range: <built-in function range>) ['$9.1']
    $9.3 = call $9.1(size, kws=[], args=[Var(size, LLH.py:283)], func=$9.1, vararg=None) ['$9.1', '$9.3', 'size']
    $9.4 = getiter(value=$9.3)               ['$9.3', '$9.4']
    $phi19.1 = $9.4                          ['$9.4', '$phi19.1']
    jump 19                                  []
label 19:
    $19.2 = iternext(value=$phi19.1)         ['$19.2', '$phi19.1']
    $19.3 = pair_first(value=$19.2)          ['$19.2', '$19.3']
    $19.4 = pair_second(v

DEBUG:numba.interpreter:label 0:
    mu = arg(0, name=mu)                     ['mu']
    sig = arg(1, name=sig)                   ['sig']
    $const0.2 = const(int, 2)                ['$const0.2']
    $0.3 = sig ** $const0.2                  ['$0.3', '$const0.2', 'sig']
    sig2 = $0.3                              ['$0.3', 'sig2']
    $0.4 = global(np: <module 'numpy' from '/home/jimmy/miniconda3/envs/nitrates_py27/lib/python2.7/site-packages/numpy/__init__.pyc'>) ['$0.4']
    $0.5 = getattr(attr=exp, value=$0.4)     ['$0.4', '$0.5']
    $0.8 = sig2 - mu                         ['$0.8', 'mu', 'sig2']
    $const0.9 = const(int, 2)                ['$const0.9']
    $0.10 = $0.8 ** $const0.9                ['$0.10', '$0.8', '$const0.9']
    $const0.12 = const(int, 2)               ['$const0.12']
    $0.13 = mu ** $const0.12                 ['$0.13', '$const0.12', 'mu']
    $0.14 = $0.10 - $0.13                    ['$0.10', '$0.13', '$0.14']
    $const0.15 = const(float, 2.0)           ['$c

DEBUG:numba.interpreter:label 0:
    mu = arg(0, name=mu)                     ['mu']
    sig = arg(1, name=sig)                   ['sig']
    $const0.2 = const(int, 2)                ['$const0.2']
    $0.3 = sig ** $const0.2                  ['$0.3', '$const0.2', 'sig']
    sig2 = $0.3                              ['$0.3', 'sig2']
    $0.4 = global(np: <module 'numpy' from '/home/jimmy/miniconda3/envs/nitrates_py27/lib/python2.7/site-packages/numpy/__init__.pyc'>) ['$0.4']
    $0.5 = getattr(attr=exp, value=$0.4)     ['$0.4', '$0.5']
    $const0.7 = const(float, 2.0)            ['$const0.7']
    $0.8 = sig2 /? $const0.7                 ['$0.8', '$const0.7', 'sig2']
    $0.10 = $0.8 - mu                        ['$0.10', '$0.8', 'mu']
    $0.11 = call $0.5($0.10, kws=[], args=[Var($0.10, LLH.py:234)], func=$0.5, vararg=None) ['$0.10', '$0.11', '$0.5']
    eterm = $0.11                            ['$0.11', 'eterm']
    $0.14 = mu - sig2                        ['$0.14', 'mu', 'sig2']
    m

DEBUG:numba.interpreter:label 0:
    N = arg(0, name=N)                       ['N']
    $const0.1 = const(float, 1.0)            ['$const0.1']
    res = $const0.1                          ['$const0.1', 'res']
    jump 6                                   []
label 6:
    jump 9                                   []
label 9:
    $9.1 = global(range: <built-in function range>) ['$9.1']
    $const9.2 = const(int, 1)                ['$const9.2']
    $const9.4 = const(int, 1)                ['$const9.4']
    $9.5 = N + $const9.4                     ['$9.5', '$const9.4', 'N']
    $9.6 = call $9.1($const9.2, $9.5, kws=[], args=[Var($const9.2, LLH.py:258), Var($9.5, LLH.py:258)], func=$9.1, vararg=None) ['$9.1', '$9.5', '$9.6', '$const9.2']
    $9.7 = getiter(value=$9.6)               ['$9.6', '$9.7']
    $phi26.1 = $9.7                          ['$9.7', '$phi26.1']
    jump 26                                  []
label 26:
    $26.2 = iternext(value=$phi26.1)         ['$26.2', '$phi26.1']
    $26

In [20]:
print(res)
print(nllh)
print(pars)

[      fun: 45894.07068100061
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00239975])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 8
      nit: 5
   status: 0
  success: True
        x: array([0.006875])]
[45894.07068100061]
[[0.006875003655224621]]


In [21]:
%%time
pars_['Signal_A'] = 1e-10
bkg_nllh = -sig_llh_obj.get_logprob(pars_)
print bkg_nllh
sqrtTS = np.sqrt(2.*(bkg_nllh - nllh[0]))
print sqrtTS

46038.7433272
17.010152627568356
CPU times: user 14.7 ms, sys: 1.02 ms, total: 15.7 ms
Wall time: 14.7 ms
